In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px 
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import random 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim 

In [ ]:
df = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts().plot.pie(explode=[0.1,0.1],autopct='%1.1f%%',shadow=True)
plt.title('type of our data');

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.total_runs.value_counts().plot.pie(explode=None,autopct='%1.1f%%',shadow=True)
plt.title('the % of total_runs')

In [ ]:
df.hist(figsize=(15,15),edgecolor='black');


In [ ]:
df1 = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df1['season'] = df1['date'].str[:4].astype(int)
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.dtypes.value_counts().plot.pie(explode=None,autopct='%1.1f%%',shadow=True)
plt.title('type of our data');

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1.describe()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.hist(figsize=(15,15),edgecolor='black');


In [ ]:
df1.winner.value_counts().plot.pie(explode=None,autopct='%1.1f%%',shadow=True)
plt.title('the % of winner')

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(x='winner',data=df1)
plt.xticks(rotation=90)
plt.title('the ages of our persone')

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(x="venue",data=df1)
plt.xticks(rotation=90)
plt.title('the player_of_match')

In [ ]:
bat = df.groupby(['batsman','id']).agg({'ball':'count','batsman_runs':'sum'})
bat = bat.groupby(['batsman']).agg({'count','mean','sum'})
bat.columns = ['total_matches1', 'mean_balls_faced','total_balls_faced','total_matches2','mean_runs','total_runs']
bat['rate'] = (bat['total_runs']*100)/bat['total_balls_faced']
high_avg = bat[bat['total_matches1']>=30].sort_values(by='mean_runs',ascending=False)[:10]
high_rate = bat[bat['total_matches1']>=30].sort_values(by='rate',ascending=False)[:10]
fig = make_subplots(rows=2, cols=1, subplot_titles=('Highest Strike Rate','Highest Average'))
fig.add_trace(
    go.Scatter(x=high_rate.index, y=high_rate['rate'], mode='markers', marker=dict(size=10,color = '#f79902'),line=dict(color='#000000',width=15), name = 'Strike rate'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=high_avg.index, y=high_avg['mean_runs'], mode='markers', marker=dict(size=10,color = '#7d1dde'),line=dict(color='#000000',width=15), name = 'Average'),
    row=2, col=1)

iplot(fig)

In [ ]:
print(df.shape)
print(df1.shape)


In [ ]:
data = pd.merge(left=df1, right=df, on='id', how='right')
data.head()

In [ ]:
print(data.shape)

In [ ]:
data.info()

# Extracting year from the date

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = pd.DatetimeIndex(data['date']).year

In [ ]:
data.head()

# Runs scored over the years

In [ ]:
runs_by_years = data.groupby(by='year').sum()['total_runs']
runs_by_years = pd.DataFrame(runs_by_years)
runs_by_years.reset_index(inplace=True)

In [ ]:
total_runs = go.Scatter(
                    x=runs_by_years['year'],
                    y=runs_by_years['total_runs'],
                    mode='lines',
                    name='runs')

data = [total_runs]

layout = go.Layout(title='Runs scored by year',
                  xaxis = dict(title='Year'),
                  yaxis = dict(title='Runs'))

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


# Preferred toss decision

In [ ]:
toss_decisions = df1.groupby(by='toss_decision').count()
toss_decisions = pd.DataFrame(toss_decisions['id'])
toss_decisions.reset_index(inplace=True)
toss_decisions

In [ ]:
toss_decision = go.Bar(
                    x=toss_decisions['toss_decision'],
                    y=toss_decisions['id']
                )

data = [toss_decision]

layout = go.Layout(title='Toss decision',
                  xaxis = dict(title='Decision'),
                  yaxis = dict(title='count'))

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)

# Totals runs and wickets by over

In [ ]:
runs_and_wickets_by_over = df.groupby(by='over').sum()
runs_and_wickets_by_over = pd.DataFrame(runs_and_wickets_by_over[['total_runs', 'is_wicket']])
runs_and_wickets_by_over.reset_index(inplace=True)
runs_and_wickets_by_over

In [ ]:
runs_and_wickets_by_overs = go.Scatter(
                    x=runs_and_wickets_by_over['over'],
                    y=runs_and_wickets_by_over['total_runs'],
                    text=runs_and_wickets_by_over['is_wicket'],
                    mode='markers',
                    marker=dict(size=runs_and_wickets_by_over['is_wicket']/10,
                               color=runs_and_wickets_by_over['total_runs']/10,
                               showscale=True)
                )

data = [runs_and_wickets_by_overs]

layout = go.Layout(title='Runs and wicket by over',
                  xaxis = dict(title='Over'),
                  yaxis = dict(title='Runs'))

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)
